In [1]:
# Import SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, ArrayType, FloatType, StructType, StructField, IntegerType
from pyspark.sql.functions import spark_partition_id, col

In [2]:
## the distance function
from scipy.spatial import distance
## creates the feature vector
from pyspark.ml.feature import VectorAssembler
## import numpy
import numpy as np

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import max, abs, min, when


In [3]:
## start session
spark = SparkSession.builder.appName("SparkRandomCoordinatesDataset").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 23:58:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
def simpleF(oq):
    return F.udf(lambda x: float(distance.euclidean(x, oq)), DoubleType())

##DEFINES A UDF FROM L2DIST THE DATAFRAME FROM A QUERY SET
distance_udf = F.udf(lambda x,y: float(distance.euclidean(x, y)), DoubleType())

In [6]:
## defines query object and 
oq = [np.random.rand(1)[0],np.random.rand(1)[0]]

#oq = [5.0,5.0]

k = 10


pivot = [np.random.rand(1)[0],np.random.rand(1)[0]]

#pivots = [(i+1,[np.random.rand(1).round(5)[0],np.random.rand(1).round(5)[0]]) for i in range(2)]
#pvt1 = [0.0,0.0]

dist_pivot_oq = distance.euclidean(pivot,oq) 

#pvtColumns = ["idPvt","fvPvt"]
#pvtDF = spark.createDataFrame(data=pivots, schema = pvtColumns)

In [7]:
## load the dataset (local)

#data = [(1,0.12,0.25),(2,0.85,0.75),(3,0.18,0.15),(4,8.1,7.1),(5,5.6,6.9),(6,3.6,4.9),(7,2.1,4.9),(8,8.6,3.9)]

#schema = ["id","coord_x","coord_y"]

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("coord_x", FloatType(), True),
    StructField("coord_y", FloatType(), True)])


#df = spark.createDataFrame(data = data, schema = schema)
df = spark.read.csv("/home/weber/Documents/coordDF2.csv",schema=schema)

##SMALL SANITY CHECK - @PRODUCTION TESTAR LOADING
df = df.na.drop()

In [8]:
## define the struct for the dimensional feature vector
cNames = df.columns
cNames.remove("id")
assembler = VectorAssembler(
    inputCols=cNames,
    outputCol="fv")

In [9]:
## appends the fv into the dataframe as column
df = assembler.transform(df)

In [1]:
df.show()

NameError: name 'df' is not defined

In [11]:
df_new = df.withColumn(f'distances_oi_pivot', simpleF(pivot)(F.col('fv')))
#df_new.repartition(10)

In [12]:
df_with_dist = df_new.withColumn("|d(oq,p1)-d(oi,p1)|", \
    abs (dist_pvt1_oq - \
    F.col('distances_oi_pivot')))

df_with_dist = df_with_dist.orderBy("|d(oq,p1)-d(oi,p1)|")

df_2 = df_with_dist.repartitionByRange(20, col('|d(oq,p1)-d(oi,p1)|'))

+---+-------+-------+--------------------+-------------------+--------------------+
| id|coord_x|coord_y|                  fv|     distances_pvt1| |d(oq,p1)-d(oi,p1)||
+---+-------+-------+--------------------+-------------------+--------------------+
| 95|0.71852|0.57866|[0.71851998567581...|0.48866349684393884|0.006295020710815502|
|  5|0.49794|0.54987|[0.49794000387191...| 0.5042247550866347|0.009266237531880328|
| 84| 0.4196|0.53659|[0.41960000991821...| 0.5299143757171434| 0.03495585816238905|
| 23|0.33079|0.35678|[0.33079001307487...| 0.4595715198475419| 0.03538699770721243|
| 90|0.57788| 0.5282|[0.57788002490997...|0.45612942342265983| 0.03882909413209451|
| 92|0.74659|0.53865|[0.74659001827239...|0.45038234059071536| 0.04457617696403898|
| 63|0.66367|0.53602|[0.66367000341415...|0.44776357588848714|  0.0471949416662672|
|  3|0.19594|0.30892|[0.19594000279903...| 0.5541855637868418|0.059227046232087466|
| 97|0.14923|0.05248|[0.14923000335693...| 0.5571768984731591| 0.06221838091

In [13]:
#df_new = df_new.withColumn("partition_id", spark_partition_id()

def knn_laesa(dataframe, oq, k):
    
    for i in range(df_new2.rdd.getNumPartitions()):

        if i == 0:
            ## PART_ONE
            df_part0 = df_new2.where(spark_partition_id() == i).localCheckpoint()
            df_part0_new = df_part0.withColumn("dist_Oq", simpleF(oq)(F.col('fv')))
            df_part0_new = df_part0_new.orderBy("dist_Oq").limit(k)
            max_value2 = df_part0_new.agg(max('dist_Oq')).collect()[0][0]

        else:
            ## PART_TWO

            df_parti = df_new2.filter(spark_partition_id() == i).localCheckpoint()
            df_parti = df_parti.where(df_parti["|d(oq,p1)-d(oi,p1)|"] <= max_value2)
            df_parti_new = df_part1.withColumn("dist_Oq", simpleF(oq)(F.col('fv')))



            ## PART_THREE
            df_part0_new = df_part0_new.union(df_parti_new).orderBy("dist_Oq").limit(k).localCheckpoint()
            max_value2 = df_part0_new.agg(max('dist_Oq')).collect()[0][0]
    return(df_part0_new)
    

In [14]:
df_part0_new.show()

+---+-------+-------+--------------------+-------------------+--------------------+-------------------+
| id|coord_x|coord_y|                  fv|     distances_pvt1| |d(oq,p1)-d(oi,p1)||            dist_Oq|
+---+-------+-------+--------------------+-------------------+--------------------+-------------------+
| 23|0.33079|0.35678|[0.33079001307487...| 0.4595715198475419| 0.03538699770721243|  0.054694271806931|
| 41|0.37885|0.35039|[0.37885001301765...|0.41733423938085046| 0.07762427817390388|0.07906928649899292|
| 65|0.29393|0.51949|[0.29392999410629...| 0.5944682699907858|  0.0995097524360315|0.11348251135535493|
| 45|0.40363|0.31745|[0.40362998843193...| 0.3775631125213736| 0.11739540503338075|0.12026322263944784|
|  4|0.21342|0.44719|[0.21342000365257...| 0.6076469790228931| 0.11268846146813877|  0.120593966299579|
| 84| 0.4196|0.53659|[0.41960000991821...| 0.5299143757171434| 0.03495585816238905|0.15475090478026468|
| 16|0.36876|0.56382|[0.36875998973846...| 0.5809299765300822| 0

In [ ]:
# measure the execution time of the function
start_time = time.time()
result = my_function(df)
end_time = time.time()

# print the execution time
print("Execution time:", end_time - start_time)